Сравним результаты моделей

Введем несколько метрик.

Ссылаюсь на эту статью: http://habr.com/ru/articles/733492/

In [ ]:
import pandas as pd

from rapidfuzz import fuzz
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
# Загрузка данных
LLM_df = pd.read_csv('predictions_LLM.csv')
BERT_df = pd.read_csv('test_bert_ner.csv')

# Замена NaN на пустые строки
LLM_df.fillna('', inplace=True)
BERT_df.fillna('', inplace=True)

In [37]:
# Функция для расчета среднего расстояния Левенштейна
def avg_levenshtein_distance(y_true, y_pred):
    total_distance = 0
    count = 0

    for true, pred in zip(y_true, y_pred):
        distance = fuzz.ratio(true, pred) / 100  # Преобразуем в долю
        total_distance += distance
        count += 1

    return total_distance / count

In [38]:
# Функция для преобразования строк в векторы
def str_to_vecs(text, target):
    vectorizer = CountVectorizer(analyzer='char')
    vectorizer = vectorizer.fit_transform([text, target])
    return vectorizer.toarray()[0], vectorizer.toarray()[1]

# Функция для вычисления косинусного сходства
def cos_sim_vecs(vec1, vec2):
    vec1 = vec1.reshape(1, -1)
    vec2 = vec2.reshape(1, -1)
    return cosine_similarity(vec1, vec2)[0][0]

# Функция для расчета среднего косинусного сходства
def avg_cos_sim_vecs(y_true, y_pred):
    total_distance = 0
    count = 0

    for true, pred in zip(y_true, y_pred):
        vec_true, vec_pred = str_to_vecs(true, pred)
        distance = cos_sim_vecs(vec_true, vec_pred)
        total_distance += distance
        count += 1

    return total_distance / count

In [39]:
# Функция для расчета доли точных совпадений
def exact_accuracy(y_true, y_pred):
    correct_matches = sum(true == pred for true, pred in zip(y_true, y_pred))
    return correct_matches / len(y_true)

In [43]:
# Вычисление метрик для моделей
levenshtein_LLM = avg_levenshtein_distance(LLM_df['output'], LLM_df['predictions'])
levenshtein_BERT = avg_levenshtein_distance(BERT_df['entity'], BERT_df['predictions'])
print(f"Среднее расстояние Левенштейна: LLM = {levenshtein_LLM:.4f}, BERT = {levenshtein_BERT:.4f}")


Среднее расстояние Левенштейна: LLM = 0.9606, BERT = 0.9143


In [44]:
# Среднее косинусное сходство
cosine_LLM = avg_cos_sim_vecs(LLM_df['output'], LLM_df['predictions'])
cosine_BERT = avg_cos_sim_vecs(BERT_df['entity'], BERT_df['predictions'])
print(f"Среднее косинусное сходство: LLM = {cosine_LLM:.4f}, BERT = {cosine_BERT:.4f}")


Среднее косинусное сходство: LLM = 0.9685, BERT = 0.9320


In [45]:
# Доля точных совпадений
exact_LLM = exact_accuracy(LLM_df['output'], LLM_df['predictions'])
exact_BERT = exact_accuracy(BERT_df['entity'], BERT_df['predictions'])
print(f"Доля точных совпадений: LLM = {exact_LLM:.4f}, BERT = {exact_BERT:.4f}")


Доля точных совпадений: LLM = 0.9082, BERT = 0.8271


Делаем вывод, что модель на основе легкой LLM показала себя лучше, чем модель на основе тяжелого BERT.

Так как по всем метрикам она лучше. Но в свою дообучать LLM может быть затратнее и сама модель будет очень тяжелая.